```python
"""
Based heavily on nestauk/nesta/nesta/core/routines/arxiv/deepchange_analysis_task
"""
```

In [1]:
%%capture
import os
import sys
LOCAL_PATH_TO_NESTA = "/Users/jklinger/Nesta/nesta/"

sys.path.append(LOCAL_PATH_TO_NESTA)
os.environ['MYSQLDB'] = f"{LOCAL_PATH_TO_NESTA}/nesta/core/config/mysqldb.config"
from nesta.core.routines.arxiv.deepchange_analysis_task import *

In [2]:
engine = get_mysql_engine("MYSQLDB", 'mysqldb', "production")

# All queries except last prepare temporary tables                        
# and the final query produces the dataframe                              
# which collects data, such that there is one row per                     
# article / institute / institute country                                 
for query, is_last in sql_queries():
    if not is_last:
        engine.execute(query)
df = pd.read_sql(query, engine)

# Manual hack to factor Hong Kong outside of China                        
for city in ["Hong Kong", "Tsuen Wan", "Tuen Mun", "Tai Po", "Sai Kung"]:
    df.loc[df.institute_city == f"{city}, CN", "institute_country"] = "Hong Kong"

# Manual hack to factor out transnational corps                           
countries = set(df.institute_country)
df['is_multinational'] = df['institute_name'].apply(lambda x: dc.is_multinational(x, countries))
df.loc[df.is_multinational, 'institute_city'] = df.loc[df.is_multinational, 'institute_name'].apply(lambda x: ''.join(x.split("(")[:-1]))
df.loc[df.is_multinational, 'institute_country'] = "Transnationals"

# collect topics, determine which represents                              
# deep_learning and apply flag                                            
terms = ["deep", "deep_learning", "reinforcement", "neural_networks", "neural_network"]
dl_topic_ids = dc.get_article_ids_by_terms(engine, terms=terms, min_weight=0.3)
df['is_dl'] = df.article_id.apply(lambda i: i in dl_topic_ids)
df['date'] = df.apply(lambda row: row.article_updated or row.article_created, axis=1)
df['year'] = df.date.apply(lambda date: date.year)
df = dc.add_before_date_flag(df, date_column='date', before_year=YEAR_THRESHOLD)

# Group by country, pivot by year
# Note: values=0 since the new size() variable is rendered under the column called "0"
grouped = df.loc[df.is_dl].groupby(["institute_country", "year"]).size().reset_index()
pivot = pd.pivot_table(grouped, index='institute_country', columns='year', values=0).fillna(0)
pivot = pivot.astype(int)
# Calculate the total counts (don't append yet so that you do accumulative sums)
total_after_2012 = pivot.apply((lambda row: sum(value for year, value in row.items() if year >= 2012)), axis=1)
total = pivot.apply(sum, axis=1)
# Now append
pivot['total'] = total
pivot['total_after_2012'] = total_after_2012
# Save to CSV
today = dt.strftime(dt.now(), format="%Y-%m-%d")
pivot.to_csv(f'dl_counts_by_year-{today}.csv')

In [3]:
pivot.head()

year,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,...,2013,2014,2015,2016,2017,2018,2019,2020,total,total_after_2012
institute_country,,,,,,,,,,,,,,,,,,,,,
Algeria,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,1
Argentina,0,0,0,0,0,0,0,0,0,0,...,0,0,2,1,0,2,5,5,17,15
Australia,0,0,0,0,0,0,0,0,0,0,...,22,51,71,146,216,223,353,349,1476,1449
Austria,0,0,0,0,0,0,0,0,1,0,...,1,6,10,20,26,23,31,46,167,165
Bangladesh,0,0,0,0,0,0,0,0,0,0,...,1,2,1,3,1,5,6,3,23,22
